# Title
[]()

In [4]:

import sys
sys.path.append(r"/home/silvhua/custom_python")
sys.path.append(r'/home/silvhua/repositories/GHL-chat/src/')
from silvhua import *

In [ ]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', 100)
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [ ]:
path = '/home/silvhua/repositories/notion/data/'


# Iteration 1

In [ ]:
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class Chatbot_Response(BaseModel):
    response: str = Field(description="The response to the InboundMessage.")
    alert_human: bool = Field(description="Whether or not to alert a human to review the response.")

def create_system_message(
        business_name, prompts_filepath='../prompts',
        examples_filepath='../data/chat_examples', doc_filepath='../data/rag_docs'
        ):
    
    instructions = load_txt(f'{business_name}.md', prompts_filepath)
    examples = load_txt(f'{business_name}.txt', examples_filepath)
    document = load_txt(f'{business_name}_doc.md', doc_filepath)

    system_message = f"""# Stage 1

{instructions}

Return your response on a JSON format with the following keys:
- "response" (string): The response to the InboundMessage.
- "alert_human" (True or False): Whether or not to alert a human to review the response.

## Examples

Below are example conversations with leads. Each lead as a unique contact ID.
An InboundMessage is from the lead. An OutboundMessage is from you.

{examples}

## Relevant documentation

{document}

# Stage 2

Review your response from stage 1. 
Revise your response if needed to make sure you followed the instructions.
Make sure that if the question cannot be answered through the documentation, 
you return "[ALERT HUMAN]".

# Stage 3

Review your response from stage 2 to ensure your response is concise.
    """

    prompt = """
    Write the next OutboundMessage based on the following InboundMessage, 
    which is delimited by triple backticks: ```{InboundMessage}```
    """
    system_message = f'{system_message}{prompt}'
    return system_message

def create_chatbot(contactId, system_message, tools, model="gpt-3.5-turbo-1106", verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        model=model, 
        model_kwargs={"response_format": {"type": "json_object"}} # https://platform.openai.com/docs/guides/text-generation/json-mode  # https://api.python.langchain.com/en/latest/chat_models/langchain_community.chat_models.openai.ChatOpenAI.html?highlight=chatopenai#
        )
    message_history = DynamoDBChatMessageHistory(
        table_name="SessionTable", session_id=contactId,
        key={
            "SessionId": contactId,
            "type": 'ChatHistory',
            }
        )
    system_message = SystemMessage(
        content=(system_message),
        input_variables=['InboundMessage']
    )

    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    parser = SimpleJsonOutputParser(pydantic_object=Chatbot_Response)
    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor( # Example of creating a custom agent: https://python.langchain.com/docs/modules/agents/how_to/custom_agent
        agent=agent, tools=tools, 
        verbose=verbose, return_intermediate_steps=True,
        parser=parser  # Add the parser instance to the agent_executor
        )
    agent_info = {
        'agent': agent,
        'agent_executor': agent_executor,
        'chat_history': message_history.messages
    }
    return agent_info

def chat_with_chatbot(user_input, agent_info):
    start_time = time()
    print(f'Chat history length: {len(agent_info["chat_history"])}')
    chat_history = agent_info['chat_history']
    result = agent_info['agent_executor']({
            "input": user_input,
            "chat_history": chat_history
        })  
    print(f'Response time: {time() - start_time} seconds')
    
    return result

def alert_human(str) -> str:
    return {"response": "[ALERT HUMAN]", "alert_human": True}

def placeholder_function(str):
    return None

tools = [
    # Tool(
    #     name=f"alert_human_function",
    #     func=alert_human,
    #     description=f"If the InboundMessage cannot be answered from your instructions or documentation, use this function.",
    # ),
    Tool(
        name=f"placeholder_function",
        func=placeholder_function,
        description=f"This function does not do anything.",
    )
]

conversation_id = 9.1
InboundMessage = "Tell me more about the program"
reply_dict[conversation_id] = dict()
question_id = 1

contactId = contacts['me_mcloone']
system_message_dict[conversation_id] = create_system_message(
    'CoachMcloone', prompts_filepath='../prompts',
    examples_filepath='../data/chat_examples', doc_filepath='../data/rag_docs'
)
conversation_dict[conversation_id] = create_chatbot(
    contactId, system_message_dict[conversation_id], tools=tools,
    # model='gpt-4-32k'
    )

reply_dict[conversation_id][question_id] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)
reply_text = reply_dict[conversation_id][question_id]["output"]
print(f'Reply from `chat_with_chatbot`: {reply_text}')


Chat history length: 2


> Entering new AgentExecutor chain...

Invoking: `placeholder_function` with `Tell me more about the program`


None
  {"response": "I'm Amanda - creator of the Strong and Sassy programme. I am a degree qualified nutritionist and coach with over 10 years experience. I educate women about the importance of health and wellness, so they can live a sustainable, healthy and balanced lifestyle. The Strong and Sassy programme includes 1:1 nutrition habit coaching, personalized resistance training programs, weekly group coaching calls, in-house mindset coaching, nutrition education, unlimited accountability, community meet-ups, and free recipe e-books.", "alert_human": false}

> Finished chain.
Response time: 7.767297029495239 seconds
Reply from `chat_with_chatbot`: 
  {"response": "I'm Amanda - creator of the Strong and Sassy programme. I am a degree qualified nutritionist and coach with over 10 years experience. I educate women about the importance of health and well

# *End of Page*